## 휴먼지능정보공학과 201810808 정민지
### 13주 과제
블록체인에서 주문을 하고, 그 내역을 로컬 파일에 작성해 보자.

블록체인으로 주문을 전송하면, 이벤트가 발생하고 주문내역을 로컬파일에 쓰게 된다.

* EventTest.sol에서는 이벤트가 발생하면서 그 로그를 파일에 작성한다 (OrderEvent.txt)

* OrderEvent.sol을 수정하여:

	* order(상품항목, 주문 개수, 배송지) 함수 작성

	* getBalance() 함수 작성. 해당 프로그램 생성한 Owner만 읽을 수 있게 modfier isOnwer 작성.



geth 또는 ganache 어느 하나를 선택하여 프로그램을 작성하세요.

아래 로그가 파일에 쓰여지도록 하고, 노트북에서 그 파일을 출력하세요 (!type src/OrderEvent.txt)

전송자주소, 1111, 3, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 5, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 20, 20 2-gil Hongji-dong Jongro-gu Seoul

In [11]:
%%writefile src/EventTestWsUse.js
var Web3=require('web3');
var fs=require('fs');
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var _abiArray  =[{"constant":false,"inputs":[],"name":"myFunction","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"my","type":"string"}],"name":"MyLog","type":"event"}];

var _test = new web3.eth.Contract(_abiArray, '0x55cDDaC70da0930fB989a02904e26DedB3264548');
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        log = JSON.stringify(result.returnValues);
        console.log("Event fired: " + log);
        fs.appendFile("src/OrderEvent.txt", log, "utf-8", function(e) {
            if(!e) {
                console.log(">> Writing to file");
            }
        //process.exit(1);
        });
    }
});

async function doIt() {    
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1);
}

doIt()

Overwriting src/EventTestWsUse.js


In [12]:
!node src/EventTestWsUse.js

Event fired: {"0":"Hello World!","my":"Hello World!"}
Account: 0xE773F79685Da7822Ae8cdC427427d3a307f0F935
>> Writing to file
Balance before: 99989536038999999800
Event fired: {"0":"Hello World!","my":"Hello World!"}
>> Writing to file
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 99989513221999999800
Balance diff: 22816999997440


In [13]:
!type src\OrderEvent.txt

{"0":"Hello World!","my":"Hello World!"}{"0":"Hello World!","my":"Hello World!"}


In [1]:
%%writefile src/OrderEvent.sol
pragma solidity 0.5.1;

contract OrderEvent {
    uint unitPrice = 10;
    address payable owner;
    event OrderLog(address indexed _from, bytes2 indexed _itemId, uint256 indexed quantity, string addr);
    
    constructor() public {
        owner = msg.sender;
    }
    
    function order(bytes2 _itemId, uint256 quantity,string memory addr) public {
        if (msg.sender == owner) {
            emit OrderLog(msg.sender, _itemId, quantity,addr);
        }
    }
    
    modifier isOwner() { //이 경우만 특정 함수가 실행되게끔, if문 줄어들게 할수있다.
        if (msg.sender != owner) {
            revert();
        }
        _; //continue executing rest of method body
    }
    
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }
}

Overwriting src/OrderEvent.sol


In [3]:
%%writefile src/OrderEvent.sol
pragma solidity 0.5.1;
contract OrderEvent {
    uint unitPrice = 10;
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value);

    function order(bytes2 _itemId, uint quantity) public payable {
        uint256 orderTime = now;
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value);
    }
}

Overwriting src/OrderEvent.sol


In [4]:
!solc --abi --bin --gas src/OrderEvent.sol


======= src/OrderEvent.sol:OrderEvent =======
Gas estimation:
construction:
   20159 + 106800 = 126959
external:
   order(bytes2,uint256):	5012
Binary: 
6080604052600a60005534801561001557600080fd5b50610216806100256000396000f3fe60806040526004361061003b576000357c01000000000000000000000000000000000000000000000000000000009004806332af7f7314610040575b600080fd5b6100976004803603604081101561005657600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190505050610099565b005b6000429050600080548302905080341415156100b457600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f1f7c3420c50f5b8875456d867e023a6d440ef2c170

In [5]:
%%writefile src/OrderEventDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantity","type":"uint256"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":true,"name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"}];
var _bin="0x" + "6080604052600a60005534801561001557600080fd5b50610216806100256000396000f3fe60806040526004361061003b576000357c01000000000000000000000000000000000000000000000000000000009004806332af7f7314610040575b600080fd5b6100976004803603604081101561005657600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190505050610099565b005b6000429050600080548302905080341415156100b457600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f1f7c3420c50f5b8875456d867e023a6d440ef2c1706d52ba38f5c6f553b53d508660405180827dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200191505060405180910390a35050505056fea165627a7a72305820b8bda7e28116845faab2ecd8ec7998c4de41a9c4b97c909c010d63da31fc52570029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 300000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })

    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/OrderEventDeploy.js


In [6]:
!node src/OrderEventDeploy.js

Deploying the contract from 0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d
hash: 0xd6f9bdac3ef4ac08050f0469f2645bee8844dbd6ae85ebe77744f4a266d44fdb
---> The contract deployed to: 0x5A91e479Ab983CE38a6B0BC9343817740c543A2E


In [7]:
!geth --exec "eth.getTransactionReceipt('0xd6f9bdac3ef4ac08050f0469f2645bee8844dbd6ae85ebe77744f4a266d44fdb')" attach http://localhost:8345

{
  blockHash: "0x985b17c9a3413d31fa1c4a82417fc49eb999fa44a6c3632f1fd6f613ad0fae37",
  blockNumber: 5,
  contractAddress: "0x5a91e479ab983ce38a6b0bc9343817740c543a2e",
  cumulativeGasUsed: 188219,
  from: "0x75cf5263f5a28c02a2c1910339e1848f9cc5e10d",
  gasUsed: 188219,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0xd6f9bdac3ef4ac08050f0469f2645bee8844dbd6ae85ebe77744f4a266d44fdb",
  transactionIndex: 0
}


In [6]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var fs=require('fs');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
}
var _abiArray = [{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantity","type":"uint256"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":true,"name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"}];
async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x5A91e479Ab983CE38a6B0BC9343817740c543A2E');
    console.log("Account: " + accounts[0]);
    const accounts = await web3.eth.getAccounts();
    var event = _order.events.OrderLog({
            filter: {_from: accounts[0], _value: 30},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            fs.appendFile("src/OrderEvent.txt", log, "utf-8", function(e) {
            if(!e) {
                console.log(">> Writing to file");
            }
        });

        }
    });
    var value;
    my = await _order.methods.order("0x1111", 3,"20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    //my = await _order.methods.order("0x1234", 4).send({from: accounts[0], gas: 100000, value:40});
    //console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    //my = await _order.methods.order("0x1234", 10).send({from: accounts[0], gas: 100000, value:100});
    //console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    process.exit(1);
}

doIt()

Overwriting src/OrderEventUse.js


In [9]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var fs=require('fs');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
}
var _abiArray = [{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantity","type":"uint256"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":true,"name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"}];
//var _abiArray=JSON.parse(_abiStr);

async function doIt(a,b,c) {
    var _order = new web3.eth.Contract(_abiArray, '0x5A91e479Ab983CE38a6B0BC9343817740c543A2E');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            //filter: {_from: accounts[0], _value: [20,50]},
            filter: {_from: accounts[0], _value: c},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            fs.appendFile("src/OrderEvent.txt", log, "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            //process.exit(1);
            });
        }
    });
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    // this will fire an event
    my = await _order.methods.order(a, b)
        .send({from: accounts[0], gas: 100000, value:c})
        //.then(function(my) {console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1);
}

doIt("0x1230",4,40)
doIt("0x1231",3,30)
doIt("0x1232",5,50)

Overwriting src/OrderEventUse.js


In [10]:
!node src/OrderEventUse.js

Account: 0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d
Account: 0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d
Account: 0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d
Balance before: 99963086839999998630
Balance before: 99963086839999998630
Balance before: 99963086839999998630
Event fired: {"0":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","1":"0x1230","2":"40","_from":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","_itemId":"0x1230","_value":"40"}
>> Writing to file
Event fired: {"0":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","1":"0x1231","2":"30","_from":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","_itemId":"0x1231","_value":"30"}
>> Writing to file
Event fired: {"0":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","1":"0x1232","2":"50","_from":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","_itemId":"0x1232","_value":"50"}
>> Writing to file
---> MyFunction called undefined
---> MyFunction called undefined
---> MyFunction called undefined
Balance after: 99961468759999998510
Balance diff: 16180800000

In [11]:
!type src\OrderEvent.txt

{"0":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","1":"0x1230","2":"40","_from":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","_itemId":"0x1230","_value":"40"}{"0":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","1":"0x1231","2":"30","_from":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","_itemId":"0x1231","_value":"30"}{"0":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","1":"0x1231","2":"30","_from":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","_itemId":"0x1231","_value":"30"}{"0":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","1":"0x1232","2":"50","_from":"0x75cF5263f5a28c02a2C1910339e1848f9Cc5E10d","_itemId":"0x1232","_value":"50"}
